In [1]:
#start

VBox()

Starting Spark application


ID,YARN Application ID,Kind,State,Spark UI,Driver log,User,Current session?
6,application_1723518592324_0008,pyspark,idle,Link,Link,user_n-renukumar_rajendran_draup_com,✔


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

SparkSession available as 'spark'.


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [2]:
sc.addPyFile("s3://jobs-emr-etl-config/jobs_run/prod/cc/ML_ETL_Config.py")
from ML_ETL_Config import *
utils_module = MlEtlUtils(spark, sc)
to_import = [name for name in dir(utils_module) if not (name.startswith('_') | (name=="spark") | (name=="sc"))]
globals().update({name: getattr(utils_module, name) for name in to_import})
from pyspark.sql.functions import *
from pyspark.sql import Window

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

404
404
/usr/local/lib/python3.9/site-packages/urllib3/connectionpool.py:1063: InsecureRequestWarning: Unverified HTTPS request is being made to host 'gateway.draup.com'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  warnings.warn(
/usr/local/lib/python3.9/site-packages/urllib3/connectionpool.py:1063: InsecureRequestWarning: Unverified HTTPS request is being made to host 'gateway.draup.com'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  warnings.warn(

In [3]:
JD_df=spark.read.format('delta').load('s3://etl-datahouse/prod/job_descriptions/etl/curated/processed_data/delta').selectExpr('data_lake_id','mvp_company_name','translated_job_title','msa','synon_location','url','job_role_list','publication_date','functional_workload','business_function','core_skills', 'soft_skills','digital_products','education','job_type','company','consolidated_jds.msa as consolidated_msa').withColumn('lower_cmp_name',lower(col('mvp_company_name')))

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [4]:
JD_df.printSchema()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

root
 |-- data_lake_id: string (nullable = true)
 |-- mvp_company_name: string (nullable = true)
 |-- translated_job_title: string (nullable = true)
 |-- msa: string (nullable = true)
 |-- synon_location: string (nullable = true)
 |-- url: string (nullable = true)
 |-- job_role_list: array (nullable = true)
 |    |-- element: string (containsNull = true)
 |-- publication_date: date (nullable = true)
 |-- functional_workload: array (nullable = true)
 |    |-- element: string (containsNull = true)
 |-- business_function: array (nullable = true)
 |    |-- element: string (containsNull = true)
 |-- core_skills: array (nullable = true)
 |    |-- element: string (containsNull = true)
 |-- soft_skills: array (nullable = true)
 |    |-- element: string (containsNull = true)
 |-- digital_products: array (nullable = true)
 |    |-- element: string (containsNull = true)
 |-- education: string (nullable = true)
 |-- job_type: struct (nullable = true)
 |    |-- job_time_based_tags: array (nullable 

In [ ]:
### Filter year

In [5]:
JD_data = JD_df.filter((col('publication_date') >= '2021-01-01') & (col('publication_date') <= '2024-08-06'))


VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

### Role input file

In [6]:
Role_df = spark.read.csv("s3://draup-command-center/temp/cc/Renukumar_Draup/karthika/Input/UK_job role_06-08-2024.csv",header = True)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [7]:
Role_df.columns

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

['Roles']

In [8]:
explode_df = JD_data.withColumn('Roles', explode(col('job_role_list')))


VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [9]:
Role_tag = explode_df.join(Role_df, ['Roles'], 'inner')

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

### company name input file

In [10]:
com_df = spark.read.csv('s3://draup-command-center/temp/cc/Renukumar_Draup/karthika/output/dpd_UK_Draup_company_name_datas_06-08-2024.csv',header = True).drop('_c0')

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [11]:
com_df.columns

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

['Draup Name', 'mvp_company_name', 'lower_company']

In [12]:
com_tag = Role_tag.join(com_df, ["mvp_company_name"],'inner')

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

### msa input file

In [13]:
loc_df=spark.read.csv('s3://draup-command-center/temp/cc/Renukumar_Draup/karthika/Input/UK_location_06-08-2024.csv',header=True)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [14]:
loc_df.columns

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

['Location', 'Country', 'Region']

In [15]:
main_df = com_tag.join(loc_df, com_tag['msa'] == loc_df['Location'], 'inner').drop('Location')

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [16]:
df_with_array = main_df.withColumn("skills_array", split(col("core_skills"), "; "))


VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [17]:
exploded_df = df_with_array.withColumn("skill", explode(col("skills_array")))


VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [22]:
result_df1 = exploded_df.groupBy('mvp_company_name','Roles','skill','publication_date').count().orderBy(desc('count'))

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [23]:
result_df1.columns

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

['mvp_company_name', 'Roles', 'skill', 'publication_date', 'count']

In [24]:
result_df1.count()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

12698

In [ ]:
from io import StringIO # python3; python2: BytesIO import boto3
import boto3
 

bucket = 'draup-command-center' # already created on S3
csv_buffer = StringIO()
   
res2=result_df1.toPandas()
res2.to_csv(csv_buffer)
s3_resource = boto3.resource('s3') 
s3_resource.Object(bucket, 'temp/cc/Renukumar_Draup/karthika/output/JD_core_skill_2021 to 2024_data_07-08-2024.csv').put(Body=csv_buffer.getvalue())

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

{'ResponseMetadata': {'RequestId': '33CRDK477DWT7WWA', 'HostId': 'V/fx7xi7fMPo1n6pvf5BUZIaGj0r6PgL0SjqXYVXQew8+snoq94yjSwYhs7q8t76iCFFBJAk8nKreeyyfPQdsw==', 'HTTPStatusCode': 200, 'HTTPHeaders': {'x-amz-id-2': 'V/fx7xi7fMPo1n6pvf5BUZIaGj0r6PgL0SjqXYVXQew8+snoq94yjSwYhs7q8t76iCFFBJAk8nKreeyyfPQdsw==', 'x-amz-request-id': '33CRDK477DWT7WWA', 'date': 'Wed, 07 Aug 2024 13:49:10 GMT', 'x-amz-expiration': 'expiry-date="Tue, 13 Aug 2024 00:00:00 GMT", rule-id="Delete in temp folder"', 'x-amz-server-side-encryption': 'AES256', 'etag': '"ac1c58dc239bf9d5fb6d5905393716fb"', 'server': 'AmazonS3', 'content-length': '0'}, 'RetryAttempts': 1}, 'Expiration': 'expiry-date="Tue, 13 Aug 2024 00:00:00 GMT", rule-id="Delete in temp folder"', 'ETag': '"ac1c58dc239bf9d5fb6d5905393716fb"', 'ServerSideEncryption': 'AES256'}

In [ ]:
## sample code

In [ ]:
skill_df = exploded_df.withColumn(
    'year',
    F.when((col('publication_date') >= '2021-01-01') & (col('publication_date') <= '2021-12-31'), '2021')
     .when((col('publication_date') >= '2022-01-01') & (col('publication_date') <= '2022-12-31'), '2022')
     .when((col('publication_date') >= '2023-01-01') & (col('publication_date') <= '2023-12-31'), '2023')
     .when((col('publication_date') >= '2024-01-01') & (col('publication_date') <= '2024-08-07'), '2024')
)

In [ ]:
skills_2021 = exploded_df.filter((col('publication_date') >= '2021-01-01') & (col('publication_date') <= '2021-12-31')).groupBy('skill').count().withColumnRenamed("count", "skills-2021-count")
skills_2022 = exploded_df.filter((col('publication_date') >= '2022-01-01') & (col('publication_date') <= '2022-12-31')).groupBy('skill').count().withColumnRenamed("count", "skills-2022-count")
skills_2023 = exploded_df.filter((col('publication_date') >= '2023-01-01') & (col('publication_date') <= '2023-12-31')).groupBy('skill').count().withColumnRenamed("count", "skills-2023-count")
skills_2024 = exploded_df.filter((col('publication_date') >= '2024-01-01') & (col('publication_date') <= '2024-08-06')).groupBy('skill').count().withColumnRenamed("count", "skills-2024-count")

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [ ]:
result_df = exploded_df \
    .join(skills_2021, on="skill", how="left") \
    .join(skills_2022, on="skill", how="left") \
    .join(skills_2023, on="skill", how="left") \
    .join(skills_2024, on="skill", how="left")

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [ ]:
result_df.columns

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

['skill', 'mvp_company_name', 'Roles', 'data_lake_id', 'translated_job_title', 'msa', 'synon_location', 'url', 'job_role_list', 'publication_date', 'functional_workload', 'business_function', 'core_skills', 'soft_skills', 'digital_products', 'education', 'job_type', 'company', 'consolidated_msa', 'lower_cmp_name', 'Draup Name', 'lower_company', 'Country', 'Region', 'skills_array', 'skills-2021-count', 'skills-2022-count', 'skills-2023-count', 'skills-2024-count']

In [ ]:
result_df[["skill","skills-2021-count","skills-2022-count"]].show(truncate = False)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+-------------------------------------+-----------------+-----------------+
|skill                                |skills-2021-count|skills-2022-count|
+-------------------------------------+-----------------+-----------------+
|Broadcasting                         |468              |6396             |
|User Testing                         |NULL             |156              |
|JavaScript                           |15988            |14239            |
|Web Graphics Library (WebGL)         |NULL             |780              |
|Web Graphics Library (WebGL)         |NULL             |780              |
|Agile Methodologies                  |10065            |12315            |
|Framer                               |NULL             |156              |
|Framer                               |NULL             |156              |
|User Interface (UI) Design           |70               |479              |
|Next.js                              |NULL             |156              |
|GraphQL    

In [ ]:
result_df1 = result_df.select('mvp_company_name','Roles','skill','skills-2021-count','skills-2022-count','skills-2023-count','skills-2024-count')

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [ ]:
from io import StringIO # python3; python2: BytesIO import boto3
import boto3


bucket = 'draup-command-center' # already created on S3
csv_buffer = StringIO()
   
res2=result_df1.toPandas()
res2.to_csv(csv_buffer)
s3_resource = boto3.resource('s3') 
s3_resource.Object(bucket, 'temp/cc/Renukumar_Draup/karthika/output/JD_skills_data_07-08-2024.csv').put(Body=csv_buffer.getvalue())

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

{'ResponseMetadata': {'RequestId': 'DQV7JEXF6S4P6V1Y', 'HostId': 'w+26RDF2DxmjBHOYScSyPwHylnzCeY6JHlkr1B7VSsVJdCO+9b8/Vp3K3UG6VPQYhxuD7ZpMDhQstB82xR8zmQ==', 'HTTPStatusCode': 200, 'HTTPHeaders': {'x-amz-id-2': 'w+26RDF2DxmjBHOYScSyPwHylnzCeY6JHlkr1B7VSsVJdCO+9b8/Vp3K3UG6VPQYhxuD7ZpMDhQstB82xR8zmQ==', 'x-amz-request-id': 'DQV7JEXF6S4P6V1Y', 'date': 'Wed, 07 Aug 2024 04:46:44 GMT', 'x-amz-expiration': 'expiry-date="Tue, 13 Aug 2024 00:00:00 GMT", rule-id="Delete in temp folder"', 'x-amz-server-side-encryption': 'AES256', 'etag': '"204164bdfe97d14bda05d888c1f01bc1"', 'server': 'AmazonS3', 'content-length': '0'}, 'RetryAttempts': 1}, 'Expiration': 'expiry-date="Tue, 13 Aug 2024 00:00:00 GMT", rule-id="Delete in temp folder"', 'ETag': '"204164bdfe97d14bda05d888c1f01bc1"', 'ServerSideEncryption': 'AES256'}

### Parquet input

In [28]:
#JD_data.write.mode("overwrite").parquet('s3://draup-command-center/temp/cc/Renukumar_Draup/karthika/temp/JD_2024_data_13-08-24')

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [29]:
JD_data =spark.read.parquet('s3://draup-command-center/temp/cc/Renukumar_Draup/karthika/temp/JD_2024_data_13-08-24')

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [50]:
skill_df = JD_data.withColumn('skills', explode (col('core_skills')))

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [55]:
group_df = skill_df.groupBy("skills").count().orderBy(desc('count'))

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [56]:
group_df.show(truncate = False)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+-------------------------------+-------+
|skills                         |count  |
+-------------------------------+-------+
|Microsoft Office               |7375262|
|Marketing                      |4592587|
|Project Management             |4023385|
|Recruitment                    |2518777|
|Auditing                       |2019301|
|Python (Programming Language)  |1968741|
|Patient Care                   |1900241|
|Structured Query Language (SQL)|1892093|
|Warehousing                    |1579667|
|Microsoft Excel                |1490261|
|Business Development           |1366679|
|Data Analysis                  |1354436|
|Java                           |1238409|
|Housekeeping                   |1214901|
|Technical Support              |1100230|
|Budgeting                      |1069346|
|JavaScript                     |1062169|
|Risk Management                |1059781|
|Amazon Web Services (AWS)      |1052685|
|Data Entry                     |1036352|
+-------------------------------+-

In [51]:
role_df2 = skill_df.withColumn('job_role', explode (col('job_role_list')))

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [53]:
group_df = role_df2.groupBy('job_role','skills').count().orderBy(desc('count'))

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [54]:
group_df.show(truncate = False)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+---------------------------+------------------+------+
|job_role                   |skills            |count |
+---------------------------+------------------+------+
|Project Manager            |Project Management|703202|
|Nurse                      |Patient Care      |605974|
|Project Development Manager|Project Management|449705|
|Office Assistant           |Microsoft Office  |420813|
|IT Project Manager         |Project Management|383267|
|Program Manager            |Project Management|375220|
|Marketing Manager          |Marketing         |374047|
|Project Control Manager    |Project Management|360329|
|Warehouse Supervisor       |Warehousing       |348652|
|Administrative Assistant   |Microsoft Office  |343329|
|Warehouse Manager          |Warehousing       |340615|
|Warehouse Associate        |Warehousing       |324801|
|Office Administrator       |Microsoft Office  |324206|
|Warehouse Specialist       |Warehousing       |323042|
|Human Resources Manager    |Recruitment       |